## Simulating encounters

If this is your first time using a notebook, please ensure you have [Python installed](https://www.python.org/downloads/) and you have installed the additional dependencies with the following command: `pip install -r ./automation/requirements.txt`

First, we'll navigate to the repo's root:


In [1]:
import os

if os.path.basename(os.getcwd()) != "TheGame":
    os.chdir("..")

Next, import what you need:


In [2]:
import copy
from automation.templates.bestiary import Bestiary
from automation.simulator.deck import Deck
from automation.simulator.encounter import Encounter

Use the Bestiary to provide dictionaries for each of the combatants. We can use emojis to represent them in the combat output. The Name value will be shown in the log.

Note that files with SAMPLE in the name are designed to workshop ideas and test edge cases, not be included in the game.


In [3]:
b = Bestiary(input_files=["06_Bestiary_SAMPLE.yaml", "07_PCs.yaml"]).raw_data
c1 = b["Clubs1"]
c1.update(dict(Name="💀", id="A"))
c2 = copy.copy(b["Clubs1"])
c2.update(dict(Name="👽", id="B"))
s = b["Spider Queen"]
s.update(dict(Name="🎇", id="C"))

Initialize the encounter. We can see who is involved and resource information by looking at the encounter's `PCs`, `enemies`, or `turn_order` properties.

Note that this may differ from a real encounter because each participant has their own hand and deck. In a true encounter, a GM might manage many characters with the same deck.


In [4]:
e = Encounter(PCs=[c1, c2], Enemies=[s])
e.turn_order

[19:24][WARNING]: 💀 used 1 excess XP
[19:24][WARNING]: 👽 used 1 excess XP


[🎇
 TC       :♥️ 2 | pc.HP : 11/11
 Hand     :  02 | pc.PP : 6/6
 Deck     :  51 | pc.AP : 1/1
 Discards :  01 | RestC : 11/11,
 💀
 TC       :♥️ 5 | pc.HP : 6/6
 Hand     :  02 | pc.PP : 2/2
 Deck     :  51 | pc.AP : 2/2
 Discards :  01 | RestC : 6/6,
 👽
 TC       :♦️ 3 | pc.HP : 6/6
 Hand     :  02 | pc.PP : 2/2
 Deck     :  51 | pc.AP : 2/2
 Discards :  01 | RestC : 6/6]

Next, we can simulate a couple rounds of combat. Here, each participant will choose a Power available to them (if sufficient PP) and apply it to an enemy at random. This does not yet cover buffing powers (e.g., Shield, Lend Aid) or mind control status effects (i.e., Charmed, Enthralled).


In [5]:
e.sim_round(3)

[19:24][INFO]: 🎇 used 0/6 PP with Terrain Blink
[19:24][INFO]: ['🎇 used Terrain Blink']
[19:24][INFO]: 🎇 used 1/6 PP with Attack, Mystic Aura
[19:24][INFO]: ['🎇 used Attack, Mystic Aura', '🎇 wounded 💀 by 1: AP 1/2, HP 6/6']
[19:24][INFO]: 💀 used 0/2 PP with Attack, Weapon
[19:24][INFO]: ['💀 used Attack, Weapon', '💀 wounded 🎇 by 1: AP 0/1, HP 11/11']
[19:24][INFO]: 💀 used 1/2 PP with Shield, Self
[19:24][INFO]: ['💀 used Shield, Self']
[19:24][INFO]: 👽 used 0/2 PP with Attack, Vengeance
[19:24][INFO]: ['👽 used Attack, Vengeance', '👽 wounded 🎇 by 1: AP 0/1, HP 10/11']
[19:24][INFO]: 👽 used 1/2 PP with Shield, Self
[19:24][INFO]: ['👽 used Shield, Self', '👽 wounded 🎇 by 1: AP 0/1, HP 9/11']
[19:24][INFO]: 🎇 used 0/5 PP with Terrain Blink
[19:24][INFO]: ['🎇 used Terrain Blink']
[19:24][INFO]: 🎇 used 1/5 PP with Attack, Mystic Aura
[19:24][INFO]: ['🎇 used Attack, Mystic Aura', '🎇 wounded 💀 by 1: AP 0/2, HP 6/6']
[19:24][INFO]: 💀 used 1/1 PP with Shield, Self
[19:24][INFO]: ['💀 used Shield, Se

We can even simulate epic events.


In [7]:
e.sim_epic_event(successes_needed=3)

[19:24][INFO]: Party 1, GM 0 | 💀 Suited Hit
INFO:automation:Party 1, GM 0 | 💀 Suited Hit
[19:24][INFO]: Party 2, GM 0 | 💀 Major Success
INFO:automation:Party 2, GM 0 | 💀 Major Success
[19:24][INFO]: Party 3, GM 0 | 👽 Suited Hit
INFO:automation:Party 3, GM 0 | 👽 Suited Hit
[19:24][INFO]: Party wins after 12 total cards drawn
INFO:automation:Party wins after 12 total cards drawn


Let's see how everyone is doing.


In [8]:
e.turn_order

[🎇
 TC       :♥️ 2 | pc.HP : 5/11
 Hand     :  03 | pc.PP : 3/6
 Deck     :  45 | pc.AP : 0/1
 Discards :  06 | RestC : 11/11,
 💀
 TC       :♥️ 5 | pc.HP : 6/6
 Hand     :  03 | pc.PP : 0/2
 Deck     :  42 | pc.AP : 0/2
 Discards :  09 | RestC : 6/6,
 👽
 TC       :♦️ 3 | pc.HP : 6/6
 Hand     :  03 | pc.PP : 0/2
 Deck     :  41 | pc.AP : 0/2
 Discards :  10 | RestC : 6/6]

If you're interested in draining a specific character's resources, we can index them directly and subtract values.


In [9]:
spider_queen = e.enemies[0]
spider_queen.HP = 1
spider_queen

🎇
TC       :♥️ 2 | pc.HP : 1/11
Hand     :  03 | pc.PP : 3/6
Deck     :  45 | pc.AP : 0/1
Discards :  06 | RestC : 11/11

The only thing that can't be set directly is the deck. This has to be managed by either drawing, using a fate card, or shuffling.


In [10]:
spider_queen.draw()

♠️ Q

In [11]:
spider_queen.exchange_fate()

[19:25][INFO]: Exchanged Fate Card: ♠️ A
INFO:automation:Exchanged Fate Card: ♠️ A


In [12]:
spider_queen.shuffle(2)
spider_queen

🎇
TC       :♥️ Q | pc.HP : 1/11
Hand     :  02 | pc.PP : 3/6
Deck     :  45 | pc.AP : 0/1
Discards :  07 | RestC : 11/11

We can give some or all participants a rest.


In [13]:
e.sim_quick_rest(participants=e.PCs)  # If no participants specified, all

[19:25][INFO]: 🎇 recovered 15 HP/PP/AP during Quick Rest
INFO:automation:🎇 recovered 15 HP/PP/AP during Quick Rest
[19:25][INFO]: 💀 recovered 4 HP/PP/AP during Quick Rest
INFO:automation:💀 recovered 4 HP/PP/AP during Quick Rest
[19:25][INFO]: 👽 recovered 4 HP/PP/AP during Quick Rest
INFO:automation:👽 recovered 4 HP/PP/AP during Quick Rest


In [14]:
e.turn_order

[🎇
 TC       :♠️ K | pc.HP : 12/11
 Hand     :  03 | pc.PP : 6/6
 Deck     :  47 | pc.AP : 1/1
 Discards :  04 | RestC : 3/11,
 💀
 TC       :♥️ 5 | pc.HP : 6/6
 Hand     :  03 | pc.PP : 2/2
 Deck     :  50 | pc.AP : 2/2
 Discards :  01 | RestC : 5/6,
 👽
 TC       :♠️ T | pc.HP : 6/6
 Hand     :  03 | pc.PP : 2/2
 Deck     :  50 | pc.AP : 2/2
 Discards :  01 | RestC : 5/6]

Or a full rest.


In [15]:
e.sim_full_rest()

In [16]:
e.turn_order

[🎇
 TC       :♠️ 6 | pc.HP : 11/11
 Hand     :  03 | pc.PP : 6/6
 Deck     :  50 | pc.AP : 1/1
 Discards :  01 | RestC : 11/11,
 💀
 TC       :♦️ 6 | pc.HP : 6/6
 Hand     :  03 | pc.PP : 2/2
 Deck     :  50 | pc.AP : 2/2
 Discards :  01 | RestC : 6/6,
 👽
 TC       :♦️ A | pc.HP : 6/6
 Hand     :  04 | pc.PP : 2/2
 Deck     :  50 | pc.AP : 2/2
 Discards :  00 | RestC : 6/6]

## Logging

Much of the information we're seeing above in the `[TIME][INFO]` format is as a result of the [logger](https://www.loggly.com/ultimate-guide/python-logging-basics/) that keeps track of this information. We can adjust what we see by adjusting the log level.


In [17]:
from automation.utils import logger

logger.setLevel("DEBUG")  # Most information
# logger.setLevel('INFO') # Standard information
# logger.setLevel('WARNING') # Only problems
# logger.setLevel('CRITICAL') # No information
spider_queen.save(DR=3, attrib=["AGL", "STR"], upper_lower="lower", draw_n=2)
e.sim_round()

[19:25][DEBUG]: Drew [♠️ A, ♠️ T] vs ♠️ 6 with TR 2 at Lower Hand 2: Color Miss
DEBUG:automation:Drew [♠️ A, ♠️ T] vs ♠️ 6 with TR 2 at Lower Hand 2: Color Miss
[19:25][INFO]: 🎇 used 1/6 PP with Attack, Mystic Aura
INFO:automation:🎇 used 1/6 PP with Attack, Mystic Aura
[19:25][DEBUG]: Drew [♠️ J] vs ♦️ 6 with TR 2: Miss
DEBUG:automation:Drew [♠️ J] vs ♦️ 6 with TR 2: Miss
[19:25][INFO]: ['🎇 used Attack, Mystic Aura']
INFO:automation:['🎇 used Attack, Mystic Aura']
[19:25][INFO]: 🎇 used 0/5 PP with Terrain Blink
INFO:automation:🎇 used 0/5 PP with Terrain Blink
[19:25][DEBUG]: Drew [♦️ 4] vs ♦️ 6 with TR 2: Suited Hit
DEBUG:automation:Drew [♦️ 4] vs ♦️ 6 with TR 2: Suited Hit
[19:25][INFO]: ['🎇 used Terrain Blink', '🎇 wounded 💀 by 1: AP 1/2, HP 6/6']
INFO:automation:['🎇 used Terrain Blink', '🎇 wounded 💀 by 1: AP 1/2, HP 6/6']
[19:25][INFO]: 💀 used 1/2 PP with Shield, Self
INFO:automation:💀 used 1/2 PP with Shield, Self
[19:25][DEBUG]: Drew [♥️ K] vs ♠️ 6 with TR 4: Miss
DEBUG:automation:D

For a more detailed record of checks, saves, and rests, turn on CSV logging. This will save logs of who performed which check/save or rest and some associated values.


In [18]:
e.set_csv_logging(True)
e.sim_round()
e.sim_quick_rest()

[19:25][INFO]: 🎇 used 0/5 PP with Terrain Blink
INFO:automation:🎇 used 0/5 PP with Terrain Blink
[19:25][DEBUG]: Drew [♦️ J] vs ♦️ A with TR 2: Suited Miss
DEBUG:automation:Drew [♦️ J] vs ♦️ A with TR 2: Suited Miss
[19:25][INFO]: ['🎇 used Terrain Blink']
INFO:automation:['🎇 used Terrain Blink']
[19:25][INFO]: 🎇 used 1/5 PP with Attack, Mystic Aura
INFO:automation:🎇 used 1/5 PP with Attack, Mystic Aura
[19:25][DEBUG]: Drew [♥️ 8] vs ♦️ 6 with TR 2: Color Hit
DEBUG:automation:Drew [♥️ 8] vs ♦️ 6 with TR 2: Color Hit
[19:25][INFO]: ['🎇 used Attack, Mystic Aura', '🎇 wounded 💀 by 1: AP 0/2, HP 6/6']
INFO:automation:['🎇 used Attack, Mystic Aura', '🎇 wounded 💀 by 1: AP 0/2, HP 6/6']
[19:25][INFO]: 💀 used 0/1 PP with Attack, Vengeance
INFO:automation:💀 used 0/1 PP with Attack, Vengeance
[19:25][DEBUG]: Drew [♠️ Q] vs ♠️ 6 with TR 4: Suited Miss
DEBUG:automation:Drew [♠️ Q] vs ♠️ 6 with TR 4: Suited Miss
[19:25][INFO]: ['💀 used Attack, Vengeance']
INFO:automation:['💀 used Attack, Vengeance']
[

The following uses system commands rather than Python to look at the output. To use this data more meaningfully, try loading the data with [pandas](https://pythonbasics.org/read-csv-with-pandas/).


In [19]:
!head automation/_output/log_draws.csv

date,id,check_save,result_int,result_str,DR,type,mod,upper_lower,draw_n
04/15 08:19:24,C,check,4,Major Success,2,None,0,n,1
04/15 08:19:24,C,check,-3,Miss,2,None,0,n,1
04/15 08:19:24,A,check,3,Suited Hit,2,STR,2,n,1
04/15 08:19:24,B,save,2,Color Hit,3,CON,0,n,1
04/15 08:19:24,B,check,-3,Miss,2,STR,2,n,1
04/15 08:19:24,A,check,1,Hit,3,None,0,n,1
04/15 08:19:24,B,check,-3,Miss,3,None,0,n,1
04/15 08:19:24,A,check,3,Suited Hit,3,None,0,n,1
04/15 08:19:24,B,check,1,Hit,3,None,0,n,1


In [20]:
!head automation/_output/log_rests.csv

date,name,before_after,type,discards,hand,HP,AP,PP,RestCards
01/22 09:16:38,👽,before,quick,15,3,5,0,1,7
01/22 09:16:38,👽,after,quick,16,5,8,0,2,4
01/22 09:16:38,💀,before,quick,16,5,7,1,0,7
01/22 09:16:38,💀,after,quick,18,5,8,1,2,5
01/22 09:16:38,🎇,before,quick,8,1,1,0,5,8
01/22 09:16:38,🎇,after,quick,13,1,8,0,6,3
01/22 09:16:38,👽,before,full,6,5,8,2,2,4
01/22 09:16:38,👽,after,full,1,5,7,2,2,7
01/22 09:16:38,💀,before,full,8,5,8,2,2,5
